In [1]:
import os
import zipfile
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

In [2]:
# Set the dataset path to the extracted directory
base_dir = 'food-101/images'


In [3]:
# Image Parameters
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

In [4]:
# Data Augmentation and Preprocessing
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 20% for validation
)


In [5]:
# Training and Validation Generators
train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

Found 80800 images belonging to 101 classes.


In [6]:
val_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

Found 20200 images belonging to 101 classes.


In [7]:
# Step 2: Model Development
base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze base model

In [8]:
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.2),
    layers.Dense(101, activation='softmax')  # 101 classes in Food-101 dataset
])

In [9]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Step 3: Training
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10
)

C:\Users\akhil\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
2525/2525 ━━━━━━━━━━━━━━━━━━━━ 3764s 1s/step - accuracy: 0.3182 - loss: 2.9322 - val_accuracy: 0.4746 - val_loss: 2.1249
Epoch 2/10
2525/2525 ━━━━━━━━━━━━━━━━━━━━ 13873s 5s/step - accuracy: 0.4699 - loss: 2.1504 - val_accuracy: 0.4861 - val_loss: 2.0846
Epoch 3/10
2525/2525 ━━━━━━━━━━━━━━━━━━━━ 3664s 1s/step - accuracy: 0.4885 - loss: 2.0679 - val_accuracy: 0.5020 - val_loss: 2.0390
Epoch 4/10
2525/2525 ━━━━━━━━━━━━━━━━━━━━ 5115s 2s/step - accuracy: 0.4931 - loss: 2.0546 - val_accuracy: 0.5096 - val_loss: 2.0210
Epoch 5/10
2525/2525 ━━━━━━━━━━━━━━━━━━━━ 15184s 6s/step - accuracy: 0.5036 - loss: 2.0086 - val_accuracy: 0.5025 - val_loss: 2.0324
Epoch 6/10
2525/2525 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.5011 - loss: 2.0278

In [ ]:
# Save the model
model.save('food_classifier_model.h5')

# Step 4: Visualization of Training Results
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.show()